# Implementing Common Neighbor Analysis as a custom method

Researchers commonly wish to implement their own custom analysis methods for particle simulations.
Here, we show an example of how to write Common Neighbor Analysis [(Honeycutt and Andersen, J. Phys. Chem. 91, 4950)](https://pubs.acs.org/doi/abs/10.1021/j100303a014) as a custom method using `freud` and the NetworkX package.

NetworkX can be installed with `pip install networkx`.

First, we generate random points and determine which points share neighbors.

In [1]:
import freud
import numpy as np
from collections import defaultdict
from util import make_fcc


# Use a noisy face-centered cubic (fcc) system
L = 3
box, points = make_fcc(nx=4, ny=4, nz=4, noise=0.)
lc = freud.locality.LinkCell(box, 1+np.sqrt(0.5))
nl = lc.compute(box, points).nlist

# Get all sets of common neighbors.
common_neighbors = defaultdict(list)
for i, p in enumerate(points):
    for j in nl.index_j[nl.index_i == i]:
        for k in nl.index_j[nl.index_i == j]:
            if i != k:
                common_neighbors[(i, k)].append(j)

Next, we use NetworkX to build graphs of common neighbors and compute the Common Neighbor Analysis signatures.

In [2]:
import networkx as nx
from collections import Counter

diagrams = defaultdict(list)
particle_counts = defaultdict(Counter)

for (a, b), neighbors in common_neighbors.items():
    # Build up the graph of connections between the
    # common neighbors of a and b.
    g = nx.Graph()
    for i in neighbors:
        for j in set(nl.index_j[
            nl.index_i == i]).intersection(neighbors):
            g.add_edge(i, j)

    # Define the identifiers for a CNA diagram:
    # The first integer is 1 if the particles are bonded, otherwise 2
    # The second integer is the number of shared neighbors
    # The third integer is the number of bonds among shared neighbors
    # The fourth integer is an index, just to ensure uniqueness of diagrams
    diagram_type = 2-int(b in nl.index_j[nl.index_i == a])
    key = (diagram_type, len(neighbors), g.number_of_edges())
    # If we've seen any neighborhood graphs with this signature,
    # we explicitly check if the two graphs are identical to
    # determine whether to save this one. Otherwise, we add
    # the new graph immediately.
    if key in diagrams:
        isomorphs = [nx.is_isomorphic(g, h) for h in diagrams[key]]
        if any(isomorphs):
            idx = isomorphs.index(True)
        else:
            diagrams[key].append(g)
            idx = diagrams[key].index(g)
    else:
        diagrams[key].append(g)
        idx = diagrams[key].index(g)
    cna_signature = key + (idx,)
    particle_counts[a].update([cna_signature])

Looking at the counts of common neighbor signatures, we see that the first particle of the fcc structure has 12 bonds with signature $(1, 4, 2, 0)$ as we expect.

In [3]:
particle_counts[0]

Counter({(1, 4, 2, 0): 12,
         (2, 4, 4, 0): 6,
         (2, 2, 1, 0): 24,
         (2, 1, 0, 0): 12})